In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn import preprocessing
from sklearn import metrics

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB


In [ ]:
traindata=pd.read_csv('C:/Users/Thanos/Desktop/12TH SEMESTER/ΜΗΧΑΝΙΚΗ ΜΑΘΗΣΗ/PROJECT/train.csv')

In [ ]:
train_data = traindata.drop(['id'], axis = 1)

In [ ]:
color = preprocessing.LabelEncoder()
color.fit(train_data['color'])
train_data['color'] = color.transform(train_data['color'])
train_data['color'] = train_data['color']/6

In [ ]:
type = preprocessing.LabelEncoder()
type.fit(train_data['type'])
train_data['type'] = type.transform(train_data['type'])

In [ ]:
traindata = train_data.drop(['color',], axis=1)

In [ ]:
accuracy_scorer = metrics.make_scorer(metrics.accuracy_score)

In [ ]:
def prior_prob(df, Y):
    classes = sorted(list(df[Y].unique()))
    prior = []
    for i in classes:
        prior.append(len(df[df[Y]==i])/len(df))
    return prior

In [ ]:
def likelihood_gaussian(df, feat_name, feat_val, Y, label):
    feat = list(df.columns)
    df = df[df[Y]==label]
    mean, std = df[feat_name].mean(), df[feat_name].std()
    p_x_y = (1 / (np.sqrt(2 * np.pi) * std)) *  np.exp(-((feat_val-mean)**2 / (2 * std**2 )))
    return p_x_y

In [ ]:
def naive_bayes_gaussian(df, X, Y):
    # get feature names
    features = list(df.columns)[:-1]

    # calculate prior
    prior = prior_prob(df, Y)

    Y_pred = []
    # loop over every data sample
    for x in X:
        # calculate likelihood
        labels = sorted(list(df[Y].unique()))
        likelihood = [1]*len(labels)
        for j in range(len(labels)):
            for i in range(len(features)):
                likelihood[j] *= likelihood_gaussian(df, features[i], x[i], Y, labels[j])

        # calculate posterior probability (numerator only)
        post_prob = [1]*len(labels)
        for j in range(len(labels)):
            post_prob[j] = likelihood[j] * prior[j]

        Y_pred.append(np.argmax(post_prob))

    return np.array(Y_pred) 

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(traindata, test_size=0.25, random_state=0)

X_test = test.iloc[:,:-1].values
Y_test = test.iloc[:,-1].values
Y_pred = naive_bayes_gaussian(train, X=X_test, Y="type")

F1 = (metrics.f1_score(Y_test, Y_pred, average= "weighted"))
ACC1 = (metrics.accuracy_score(Y_test, Y_pred))

print("\nF1 Score is: " + str(F1))
print("\nAccuracy Score is: " + str(ACC1))

In [ ]:
#multinomial distribution for the 5th field ('color')
X2 = train_data.drop(['bone_length','rotting_flesh', 'hair_length','has_soul','type'], axis=1)

In [ ]:
X2_train, X2_test, y_train, y_test = train_test_split(X2, train_data['type'], test_size = 0.25, random_state = 0)

In [ ]:
#Naive-Bayes(Multinomial)
clf = MultinomialNB() 
clf.fit(X2_train, y_train)
y2_pred = clf.predict(X2_test)
F2 = (metrics.f1_score(y_test, y2_pred, average= "weighted"))
ACC2 = (metrics.accuracy_score(y_test, y2_pred))

print("\nF1 Score is: " + str(F2))
print("\nAccuracy Score is: " + str(ACC2))